In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf

import vocab
from seq2seq_model import Seq2Seq

import warnings
warnings.filterwarnings("ignore")
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
tf.__version__
tf.reset_default_graph()

In [3]:
from collections import namedtuple

# params
HParams = namedtuple(
  "HParams",
  [ "cell",
    "batch_size",
    "layers",
    "attention",
    "source_vocab_path",
    "target_vocab_path",
    "rl_training",  
    "enc_embedding_dim",
    "dec_embedding_dim",
    "hidden_size",
    "attn_size",
    "eval_batch_size",
    "learning_rate",
    "max_source_len",
    "max_target_len",
    "optimizer",
    "optimizer_clip_gradients"])

# create params
def create_hparams():
    return HParams(
        cell=tf.contrib.rnn.LSTMCell,
        batch_size=32,
        rl_training=False,
        source_vocab_path='./data/vocab_en.txt',
        target_vocab_path='./data/vocab_kr.txt',
        layers=8,
        eval_batch_size=1,
        attention=True,
        optimizer="Adam",
        optimizer_clip_gradients=10.0,
        learning_rate=0.001,
        enc_embedding_dim=128,
        dec_embedding_dim=128,
        hidden_size=256,
        attn_size=256,
        max_source_len=40,
        max_target_len=40)
hparams = create_hparams()

In [4]:
#vocab info loading
source_vocab_info = vocab.get_vocab_info(hparams.source_vocab_path)
target_vocab_info = vocab.get_vocab_info(hparams.target_vocab_path)
print(source_vocab_info)
print(target_vocab_info.special_vocab.SEQUENCE_END)

VocabInfo(path='./data/vocab_en.txt', vocab_size=10124, special_vocab=SpecialVocab(UNK=10124, SEQUENCE_START=10125, SEQUENCE_END=10126))
12413


## input_fn 생성하기

In [5]:
from input_fn import create_input_fn

In [6]:
input_fn = create_input_fn(source_file_list=['data/kaist_corpus_bpe.en'],
                           target_file_list=['data/kaist_corpus_bpe.kr'],
                           batch_size=hparams.batch_size)
                           
#features, labels = input_fn()


In [7]:
## check input_fn
# loading vocab
# 데이타 체크
def callInputFnOnce(input_fn, session): 
    features, label = input_fn()
    coord =  tf.train.Coordinator()
    threads = tf.train.start_queue_runners(session, coord=coord)
    result_values = session.run([features, label])
    coord.request_stop()
    coord.join(threads)
    return result_values

print(callInputFnOnce(input_fn, tf.Session()))


[{'source_len': array([ 6,  3,  6,  7,  8,  7,  7,  6,  5,  4,  8,  6,  5,  4, 14,  5,  7,
        6,  7,  9,  5,  6,  7,  7, 10,  5,  6,  5,  6,  7,  5,  4], dtype=int32), 'source_tokens': array([[b'He', b'is', b'at', b'the', b'school.', b'SEQUENCE_END', b'', b'',
        b'', b'', b'', b'', b'', b''],
       [b'Take', b'care.', b'SEQUENCE_END', b'', b'', b'', b'', b'', b'',
        b'', b'', b'', b'', b''],
       [b'P@@', b'or@@', b't', b'arms', b'!', b'SEQUENCE_END', b'', b'',
        b'', b'', b'', b'', b'', b''],
       [b'He', b'th@@', b'undered', b'at', b'the', b'door.',
        b'SEQUENCE_END', b'', b'', b'', b'', b'', b'', b''],
       [b'There', b'is', b'a', b'great', b'need', b'of', b'money.',
        b'SEQUENCE_END', b'', b'', b'', b'', b'', b''],
       [b'He', b'is', b'ten', b'years', b'old', b'.', b'SEQUENCE_END', b'',
        b'', b'', b'', b'', b'', b''],
       [b'P@@', b'or@@', b't@@', b'land', b'B@@', b'bill.',
        b'SEQUENCE_END', b'', b'', b'', b'', b'', b'',

## seq2seq 모델생성


In [8]:
# seq2seq모델 생성
seq2seq = Seq2Seq(target_vocab_info.special_vocab.SEQUENCE_START, 
                  target_vocab_info.special_vocab.SEQUENCE_END,
                  hparams)

In [11]:
seq2seq_estimator = tf.contrib.learn.Estimator(
    model_fn=seq2seq.create_model_fn(), params=hparams, model_dir="./model_test/")
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8ec0381a58>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}


In [9]:
with tf.Session() as sess:
    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8f81d9bb38>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}


In [10]:
#training...
seq2seq_estimator.fit(input_fn=input_fn)

INFO:tensorflow:Creating vocabulary lookup table of size 10127
INFO:tensorflow:Creating vocabulary lookup table of size 12414
Tensor("Decoder/cond/PyFunc:0", dtype=float32)


ValueError: Unexpected shape <unknown> for Decoder/cond/Merge_2:0.

In [ ]:
def infer_input_fn(source_seq):
    source_seq = [source_seq.split()]
    source_seq_len = len(source_seq)
    source_seq = tf.constant(source_seq, dtype=tf.string)
    source_seq_len = tf.constant([source_seq_len], dtype=tf.int32)
        
    features = {"source_tokens": source_seq, "source_len": source_seq_len}
    print(features)
    return features, None

## RLEstimator 생성

In [ ]:
predictions = seq2seq.predict(input_fn=lambda :infer_input_fn("i am boy"))
for i, p in enumerate(predictions):
    print(p)
    predicted_tokens = np.char.decode(p.astype("S"), "utf-8")
    print(' '.join(predicted_tokens))